# Scrape Understat Website
This code creates a function to scrape an entire season of fixtures, results and xG stats for one league for one season <br>
This function will be used in future pieces of code

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import re
import json
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
def season_scrape(league,season):
    '''
    This fucntion scrapes an entire season of matches from Understat.com
    League: Name of the league. Character string.
    Season: Numeric of the year the season starts.
    Updated 23/02/2020
    '''
    
    season_str = str(season) +'/' +str(season+1)[-2:]
    url_str = 'https://understat.com/league/' +league + '/' +str(season)
    response = requests.get(url_str)
    #'https://understat.com/league/')+league + '/' +str(season)
    datesData = re.search("datesData\s+=\s+JSON.parse\('([^']+)", response.text)
    decoded_string = bytes(datesData.groups()[0], 'utf-8').decode('unicode_escape')
    datesObj = json.loads(decoded_string)
    for element in datesObj:
        matchid = element['id']
        matchdate = element['datetime']
        hometeam = element['h'].get('title')
        awayteam = element['a'].get('title')
        fixture = hometeam + ' vs ' + awayteam
        homegoals = element['goals'].get('h')
        awaygoals = element['goals'].get('a')
        hxG = element['xG'].get('h')
        axG = element['xG'].get('a')
        df_match_json = pd.DataFrame({"league": [league],
                                      "season": [season_str],
                                      "matchid": [matchid],
                                      "matchdate": [matchdate],
                                      "fixture": [fixture],
                                      "hometeam": [hometeam],
                                      "awayteam": [awayteam],
                                      "homegoals": [homegoals],
                                      "awaygoals": [awaygoals],
                                      "hxG": [hxG],
                                      "axG": [axG]}
                                    )
        if element == datesObj[0]:
            df_json_season = df_match_json
        else:
            df_json_season =df_json_season.append(df_match_json,ignore_index = True)
    return df_json_season

In [4]:
#test = season_scrape("La Liga",2014)

,league,season,matchid,matchdate,fixture,hometeam,awayteam,homegoals,awaygoals,hxG,axG
0,La Liga,2014/15,5826,2014-08-23 18:00:00,Malaga vs Athletic Club,Malaga,Athletic Club,1,0,1.32107,1.14151
1,La Liga,2014/15,5827,2014-08-23 20:00:00,Sevilla vs Valencia,Sevilla,Valencia,1,1,1.17197,1.74903
2,La Liga,2014/15,5828,2014-08-23 20:00:00,Granada vs Deportivo La Coruna,Granada,Deportivo La Coruna,2,1,0.548351,0.379371
3,La Liga,2014/15,5829,2014-08-23 22:00:00,Almeria vs Espanyol,Almeria,Espanyol,1,1,0.978788,0.399306
4,La Liga,2014/15,5830,2014-08-24 18:00:00,Eibar vs Real Sociedad,Eibar,Real Sociedad,1,0,0.465398,0.975523
